## Planetary Ensemble Prediction and Error Analysis

This notebook uses the saved MLP ensemble models and the fitted StandardScaler to generate predictions for a specified time period and planet. It then calculates and visualizes the prediction errors.

In [1]:
# --- 1. SETUP AND IMPORTS ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from datetime import datetime, timedelta
import os

# Import your utility file (ensure stars_utils.py is in the same directory)
import stars_utils

print(f'TensorFlow Version: {tf.version}')

TensorFlow Version: <module 'tensorflow._api.v2.version' from '/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tensorflow/_api/v2/version/__init__.py'>


In [2]:
# --- 2. CONFIGURATION ---

# Prediction Target and Timeframe
TARGET_PLANET = 'mars' # Change this to 'jupiter', 'venus', etc. for analysis
START_DATE = datetime(2025, 1, 1) # Start prediction after training data ends
END_DATE = datetime(2035, 1, 1)
TIME_STEP = timedelta(days=7) # Weekly data for prediction

# Model and Scaler Locations
MODEL_DIR = 'models' 
MODELS_LIS = ['mm0', 'mm1', 'mm2'] 
SCALER_FILEPATH = os.path.join(MODEL_DIR, 'feature_scaler.pkl') 


# Create model directory if it doesn't exist (useful for saving)
os.makedirs(MODEL_DIR, exist_ok=True)
print(f'Configuration loaded. Target Planet: {TARGET_PLANET.capitalize()}')

Configuration loaded. Target Planet: Mars


In [3]:
# --- 3. LOAD SCALER AND MODELS ---

print('Loading assets...')

# Load the fitted scaler object
scaler = stars_utils.load_scaler(SCALER_FILEPATH)

if scaler is None:
    raise FileNotFoundError('FATAL: Scaler not loaded. Run your training script first to save it!')

# Load the ensemble models
ensemble_models = stars_utils.models_loader(MODEL_DIR, MODELS_LIS)

if not ensemble_models:
    raise ValueError('FATAL: Failed to load any models. Check MODEL_DIR and MODELS_LIS.')

print(f'Successfully loaded {len(ensemble_models)} models and the feature scaler.')

Loading assets...


AttributeError: module 'stars_utils' has no attribute 'load_scaler'

In [ ]:
# --- 4. GENERATE GROUND TRUTH AND PREPARE INPUT ---

print(f'Generating ground truth data for {TARGET_PLANET.capitalize()}...')

# Generate the Ground Truth data (actual positions)
df_planet = stars_utils.generate_planetary_ephemeris_df(
    target_planet=TARGET_PLANET, 
    start_date=START_DATE, 
    end_date=END_DATE, 
    time_step=TIME_STEP
)

# Extract the feature (Julian Date)
X_predict = df_planet[['Julian_Date']]

# Use the loaded scaler to TRANSFORM the prediction data
X_predict_scaled = scaler.transform(X_predict)

print(f'Data generated and scaled. Ready for prediction over {len(df_planet)} steps.')

In [ ]:
# --- 5. GENERATE ENSEMBLE PREDICTIONS ---

print('Generating Ensemble Predictions...')
y_pred_list = []

for i, model in enumerate(ensemble_models):
    # Predict the RA, Dec, Distance (3 targets)
    y_pred = model.predict(X_predict_scaled, verbose=0) # Suppress prediction output
    y_pred_list.append(y_pred)

# Ensemble prediction is the simple average of all model outputs
y_ensemble_pred_au = np.mean(y_pred_list, axis=0)

# --- 6. AUGMENT DATAFRAME WITH RESULTS ---

# Add Prediction columns
df_planet['RA_pred'] = y_ensemble_pred_au[:, 0]
df_planet['Dec_pred'] = y_ensemble_pred_au[:, 1]
df_planet['Distance_pred'] = y_ensemble_pred_au[:, 2]

# Add Residual (Error) columns: Actual - Predicted
df_planet['RA_Error'] = df_planet['RA_deg'] - df_planet['RA_pred']
df_planet['Dec_Error'] = df_planet['Dec_deg'] - df_planet['Dec_pred']
df_planet['Distance_Error'] = df_planet['Distance_AU'] - df_planet['Distance_pred']

print('Prediction complete. DataFrame augmented with prediction results.')

In [ ]:
# --- 7. ANALYSIS: ERROR SUMMARY ---

print('--- Error Summary ---')
print(f'Prediction Period: {START_DATE.year} to {END_DATE.year}')

# Calculate RMSE for the entire prediction period
rmse_ra = np.sqrt(np.mean(df_planet['RA_Error']**2))
rmse_dec = np.sqrt(np.mean(df_planet['Dec_Error']**2))
rmse_dist = np.sqrt(np.mean(df_planet['Distance_Error']**2))
overall_rmse = np.mean([rmse_ra, rmse_dec, rmse_dist])

print(f'Ensemble RMSE (RA): {rmse_ra:.6f} degrees')
print(f'Ensemble RMSE (Dec): {rmse_dec:.6f} degrees')
print(f'Ensemble RMSE (Distance): {rmse_dist:.6f} AU')
print(f'Overall Average RMSE: {overall_rmse:.6f}')

print('--- DataFrame Head (Actual vs. Predicted) ---')
df_planet[['Time_UTC', 'RA_deg', 'RA_pred', 'RA_Error', 'Distance_AU', 'Distance_pred']].head()

In [ ]:
# --- 8. ANALYSIS: VISUALIZING ERROR OVER TIME ---

plt.figure(figsize=(14, 6))

plt.plot(df_planet['Time_UTC'], df_planet['RA_Error'] * 3600, label='RA Error (arcseconds)') # Convert degrees to arcseconds

plt.title(f'{TARGET_PLANET.capitalize()} Right Ascension Prediction Error Over Time')
plt.xlabel('Date')
plt.ylabel('RA Error (Arcseconds)')
plt.grid(True)
plt.legend()
plt.show()

plt.figure(figsize=(14, 6))

plt.plot(df_planet['Time_UTC'], df_planet['Distance_Error'] * 1e6, label='Distance Error (Millionths of AU)') # Convert AU to Micro-AU

plt.title(f'{TARGET_PLANET.capitalize()} Distance Prediction Error Over Time')
plt.xlabel('Date')
plt.ylabel('Distance Error (μAU)')
plt.grid(True)
plt.legend()
plt.show()